In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 


def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))



customer_id = '21'
customer_name='%menswearhouse%'
client='Mens Warehouse'

In [2]:
dateszs='2001-08-11'

dateszs='2001-11-01'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
35773


# Check for Attribute bucket Values

In [3]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['color']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

33
Number of attributes: 33

Start Buckets
0 Attribute: belt_width
Number of SKUs for the attribute belt_width: 33
Start Buckets
1 Attribute: care
Number of SKUs for the attribute care: 3269
Start Buckets
2 Attribute: category
Number of SKUs for the attribute category: 3834
Start Buckets
3 Attribute: collar_style
Number of SKUs for the attribute collar_style: 579
Start Buckets
4 Attribute: color
Number of SKUs for the attribute color: 3791
Start Buckets
5 Attribute: cuff_style
Number of SKUs for the attribute cuff_style: 417
Start Buckets
6 Attribute: cufflink_shape
Number of SKUs for the attribute cufflink_shape: 21
Start Buckets
7 Attribute: features
Number of SKUs for the attribute features: 3666
Buckets Effected: 2
Number of SKUs to be wiped: 9


Start Buckets
8 Attribute: fit
Number of SKUs for the attribute fit: 1433
Start Buckets
9 Attribute: jacket_style
Number of SKUs for the attribute jacket_style: 528
Start Buckets
10 Attribute: lapel_shape
Number of SKUs for the attribute l

# Check Freetext Attributes

In [ ]:
# patterns = {
#     'appliance_cubic_feet': r'(cu ft)|(n\/a)|()',
# }

In [ ]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

# Curated

In [3]:
fit=cb[cb['attribute'].astype(str)=='fit']

In [4]:
pat='''(?i)((?:athletic.?fit|classic.?fit|extreme.?slim.?fit|executive.?fit|extreme.?slim|husky|modern.?fit|portly|regular.?fit|relaxed.?fit|skinny.?fit|slim.?fit|straight.?fit))|()'''                                

fit['m_name']=fit['name'].apply(lambda x: re_extract(pat,str(x)))
fit['m_long']=fit['long_desc'].apply(lambda x: re_extract(pat,str(x)))
fit['m_custom']=fit['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

name=fit[fit['m_name'].astype(str)!='[]']
print(len(name))

1371


In [5]:
exteme slim fit not for : - Sports shirt button-downs - camp shirt button-downs
    
    
husky for: applicable for Boys: - Dress Suits- Coats/Blazers- Dress Pants
If title/description does not call out a specific fit, curate as "Regular Fit".


modern fit: Include Regular Fit
    
    
Portly only applies to - Blazers/Suit Coats- Business/Formal Dress Suits- Dress shirts button-downs


Regular fit: For BOYS items that do not say any specific Fit on the title/description. 
    
    
Slim Fit: NOT applicable for Camp Shirt button downs Does NOT include items that call out "trim Modern fit" -- only tag Modern Fit for those

SyntaxError: invalid syntax (<ipython-input-5-e4697b676722>, line 1)

In [6]:
fit[['external_id','value','m_name']]

,external_id,value,m_name
570,TMW_6NJD_6NJE_01,"[""Classic Fit""]",[]
571,TMW_8V9G_8VRG_02,Classic Fit,[]
572,TMW_8V9G_8VRG_19,Classic Fit,[]
573,TMW_8V9H_8VRH_19,Classic Fit,[]
574,TMW_6GXA_6GXR_01,"[""Modern Fit""]",[Modern Fit]
...,...,...,...
30404,TMW_6NDY_6NDZ_01,Slim Fit,[Slim Fit]
30405,TMW_6NE0_6NE1_01,Slim Fit,[Slim Fit]
30406,TMW_6NE2_6NE3_14,Slim Fit,[Slim Fit]
30407,TMW_6NFC_6NFD_02,Slim Fit,[Slim Fit]


In [7]:
na=fit[(fit['m_name'].astype(str)=='[]')&(fit['m_long'].astype(str)=='[]')&(fit['m_custom'].astype(str)=='[]')]
print(len(na))

df=fit[(fit['m_name'].astype(str)!='[]')|(fit['m_long'].astype(str)!='[]')|(fit['m_custom'].astype(str)!='[]')]
print(len(df))

101
1408


In [8]:
df['value'].explode().value_counts()

Slim Fit                        834
Modern Fit                      281
Classic Fit                     173
Skinny Fit                       71
Executive Fit                    10
Athletic Fit                      9
["Modern Fit"]                    8
["Classic Fit","Modern Fit"]      7
Straight Fit                      6
Extreme Slim Fit                  5
n/a                               2
["Classic Fit"]                   1
["Classic Fit","Slim Fit"]        1
Name: value, dtype: int64

In [17]:
slim=df[(df['value'].astype(str)=='Slim Fit')]#['m_custom'].explode().value_counts()

pat='''(?i)(slim.?fit)|()'''
slim['names']=slim['m_name'].apply(lambda x: re_extract(pat,str(x)))
slim['longs']=slim['m_long'].apply(lambda x: re_extract(pat,str(x)))
slim['customs']=slim['m_custom'].apply(lambda x: re_extract(pat,str(x)))

In [21]:
s=slim[(slim['names'].astype(str)=='[]')&(slim['longs'].astype(str)=='[]')&(slim['customs'].astype(str)=='[]')]
s['Q:fit']='[]'
match_slim_fit_wipe=s[['external_id','Q:fit']]

In [10]:
df[(df['value'].astype(str)=='Slim Fit')&(df['m_name'].astype(str)!='slim fit')]

# Uncurated

In [ ]:
# parameters
formatted_attribute = 'sustainability'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

# send to the folder for upload

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good(client, attribute,matchesna,today)   

In [23]:
attribute='fit'

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-match_slim_fit_wipe.csv',index=False) 
    
looks_good(client, attribute,match_slim_fit_wipe,today)  